# Python Data Cleaning with Pandas and NumPy
This notebook covers data cleaning techniques using Pandas and NumPy, as outlined in the [Real Python tutorial](https://realpython.com/python-data-cleaning-numpy-pandas/).

Data scientists spend a large amount of their time cleaning datasets and getting them down to a form with which they can work. In fact, a lot of data scientists argue that the initial steps of obtaining and cleaning data constitute 80% of the job.

Therefore, if you are just stepping into this field or planning to step into this field, it is important to be able to deal with messy data, whether that means missing values, inconsistent formatting, malformed records, or nonsensical outliers.

In this tutorial, we’ll leverage Python’s [pandas](https://realpython.com/pandas-python-explore-dataset/) and NumPy libraries to clean data.

We’ll cover the following:

Dropping unnecessary columns in a DataFrame
- Changing the index of a DataFrame
- Using `.str()` methods to clean columns
- Renaming columns to a more recognizable set of labels
- Skipping unnecessary rows in a CSV file

Here are the datasets that we will be using:

- [BL-Flickr-Images-Book.csv](https://github.com/realpython/python-data-cleaning/blob/master/Datasets/BL-Flickr-Images-Book.csv) – A CSV file containing information about books from the British Library
- [olympics.csv](https://github.com/realpython/python-data-cleaning/blob/master/Datasets/olympics.csv) – A CSV file summarizing the participation of all countries in the Summer and Winter Olympics
You can download the datasets from Real Python’s GitHub repository in order to follow the examples here.



# British Library data
## Importing Libraries
First, we'll import the necessary libraries.

In [1]:
import pandas as pd
import numpy as np

## 1.0 Data Cleaning

### 1.1 Dropping Columns in a DataFrame
Often, you’ll find that not all the categories of data in a dataset are useful to you. For example, you might have a dataset containing student information (name, grade, standard, parents’ names, and address) but want to focus on analyzing student grades.

In this case, the address or parents’ names categories are not important to you. Retaining these unneeded categories will take up unnecessary space and potentially also bog down runtime.

pandas provides a handy way of removing unwanted columns or rows from a DataFrame with the `drop()` function. Let’s look at a simple example where we drop a number of columns from a DataFrame.

First, let’s create a DataFrame out of the CSV file [BL-Flickr-Images-Book.csv]('https://raw.githubusercontent.com/realpython/python-data-cleaning/refs/heads/master/Datasets/BL-Flickr-Images-Book.csv'). Let's fetch the data straight from Github using `pd.read_csv()`.

In [ ]:
# load the data
df = pd.read_csv('https://raw.githubusercontent.com/realpython/python-data-cleaning/refs/heads/master/Datasets/BL-Flickr-Images-Book.csv')

The first thing to do when loading a new dataset is to understand the size of the data we are dealing with. Let's compute the number of rows and columns (i.e., the "shape" of the dataframe).

In [36]:
# compute the shape of the dataframe


When we look at the first five rows of the dataframe we see a lot of NaN values ("not a number"). Let's count how many NaNs there are in each column. If you don't know how to do this, use the resources available to you (Google, StackOverflow, etc)

In [ ]:
df

When we look at the first five entries using the head() method, we can see that a handful of columns provide ancillary information that would be helpful to the library but isn’t very descriptive of the books themselves: Edition Statement, Corporate Author, Corporate Contributors, Former owner, Engraver, Issuance type and Shelfmarks.

We can drop these columns in the following way:

In [11]:
to_drop = ['Edition Statement',
           'Corporate Author',
           'Corporate Contributors',
           'Former owner',
           'Engraver',
           'Contributors',
           'Issuance type',
           'Shelfmarks']


In [34]:
# drop the columns


In [ ]:
# verify we have dropped the columns

Alternatively, we could also remove the columns by passing them to the columns parameter directly instead of separately specifying the labels to be removed and the axis where pandas should look for the labels:

In [ ]:
# Drop the columns using the column parameter of the drop method

This syntax is more intuitive and readable. What we’re trying to do here is directly apparent.

### 1.2 Changing the Index of a DataFrame
A pandas Index extends the functionality of NumPy arrays to allow for more versatile slicing and labeling. In many cases, it is helpful to use a uniquely valued identifying field of the data as its index.

For example, in the dataset used in the previous section, it can be expected that when a librarian searches for a record, they may input the unique identifier (values in the Identifier column) for a book. Use `is_unique` on the Identifier column

In [ ]:
# check that the Identifier column contains only unique elements, using is_unique()

Let’s replace the existing index with this column using `set_index`:

In [ ]:
# replace the index using .set_index()

We can access each record in a straightforward way with `loc[]`. Although `loc[]` may not have all that intuitive of a name, it allows us to do label-based indexing, which is the labeling of a row or record without regard to its position:

In [33]:
# access index 206


In other words, 206 is the first label of the index. To access it by position, we could use df.iloc[0], which does position-based indexing.

Previously, our index was a RangeIndex: integers starting from 0, analogous to Python’s built-in range. By passing a column name to set_index, we have changed the index to the values in Identifier.

You may have noticed that we reassigned the variable to the object returned by the method with df = df.set_index(...). This is because, by default, the method returns a modified copy of our object and does not make the changes directly to the object. We can avoid this by setting the inplace parameter:

### ✏️ Now it's your turn. 
> Try to replicate what we have done together. 
> 1. Drop the columns Edition Statement, Corporate Author, Corporate Contributors, Former owner, Engraver, Issuance type and Shelfmarks from the dataframe df.
> 2. Change the index of the dataframe to the Identifier column.

### 1.3 Tidying up values and enforce data types
So far, we have removed unnecessary columns and changed the index of our DataFrame to something more sensible. In this section, we will clean specific columns and get them to a uniform format to get a better understanding of the dataset and enforce consistency. In particular, we will be cleaning Date of Publication and Place of Publication.

Upon inspection, all of the data types are currently the object dtype, which is roughly analogous to str in native Python.

It encapsulates any field that can’t be neatly fit as numerical or categorical data. This makes sense since we’re working with data that is initially a bunch of messy strings:

In [ ]:
# check out the type of each column
df.dtypes

Place of Publication    object
Date of Publication     object
Publisher               object
Title                   object
Author                  object
Flickr URL              object
dtype: object

One field where it makes sense to enforce a numeric value is the date of publication so that we can do calculations down the road:

In [41]:
df.loc[:,'Date of Publication']

Identifier
206        1879 [1878]
216               1868
218               1869
472               1851
480               1857
              ...     
4158088           1838
4158128       1831, 32
4159563      [1806]-22
4159587           1834
4160339        1834-43
Name: Date of Publication, Length: 8287, dtype: object

A particular book can have only one date of publication. Therefore, we need to do the following:

- Remove the extra dates in square brackets, wherever present: 1879 [1878]
- Convert date ranges to their “start date”, wherever present: 1860-63; 1839, 38-54
- Completely remove the dates we are not certain about and replace them with NumPy’s NaN: [1897?]
- Convert the string nan to NumPy’s NaN value
- Synthesizing these patterns, we can actually take advantage of a single regular expression to extract the publication year:

In [42]:
# Use this regex pattern
regex = r'^(\d{4})'

The regular expression above is meant to find any four digits at the beginning of a string, which suffices for our case. The above is a raw string (meaning that a backslash is no longer an escape character), which is standard practice with regular expressions.

The \d represents any digit, and {4} repeats this rule four times. The ^ character matches the start of a string, and the parentheses denote a capturing group, which signals to pandas that we want to extract that part of the regex. (We want ^ to avoid cases where [ starts off the string.)

Read more about regular expressions [here](https://realpython.com/regex-python/).

Let’s see what happens when we run this regex across our dataset:

In [43]:
extr = df['Date of Publication'].str.extract(regex, expand=False)
extr

Identifier
206        1879
216        1868
218        1869
472        1851
480        1857
           ... 
4158088    1838
4158128    1831
4159563     NaN
4159587    1834
4160339    1834
Name: Date of Publication, Length: 8287, dtype: object

Technically, this column still has object dtype, but we can easily get its numerical version with `pd.to_numeric`:

In [ ]:
# Convert the extracted column to numeric and assign it to the column `Date of Publication`

# verify that the type is numeric



This results in about one in every ten values being missing, which is a small price to pay for now being able to do computations on the remaining valid values:

In [ ]:
# show the proportion (%) of null values in the newly created date of publication.


'0.12'

### 1.4 Imputation: Handling missing data
You might recall that Date of Publication had some missing values

In [ ]:
df['Date of Publication']

Let's count how many exactly

In [ ]:
# count how many missing values


Missing values can break some functions. Imputation in data science is used to "fill" those missing cells with neutral values. Here are some options for imputation: 
- zero
- mean
- median

Let's start from filling the missing values with zeros. 

In [ ]:
# Fill the missing values with 0


Choosing the appropriate imputation method is crucial to ensure the data is still meaningful. For instance, filling the Date of Publication missing values with zero might create problems in their interpretation: Does zero means a value was missing or does it mean the book was published in the year 0?

In [ ]:
# Re-introduce the nan values
df.loc[df["Date of Publication"]==0, "Date of Publication"] = np.nan

Series([], Name: Date of Publication, dtype: float64)

A more appropriate method in this case is to simply drop the rows where date of publication is missing. 

In [ ]:
# drop all rows where date of publication is missing

### ✏️ Now it's your turn. 
> Try to replicate what we have done together. 
> 1. Tidy up the date of publication column using the regex above and convert it to numeric dtype.
> 2. Impute the missing values using the median date of publication. 

### 1.5 Combining `str` methods with NumPy to clean string columns
Above, you may have noticed the use of `df['Date of Publication'].str`. This attribute is a way to access speedy string operations in pandas that largely mimic operations on native Python strings or compiled regular expressions, such as `.split()`, `.replace()`, and `.capitalize()`.

To clean the Place of Publication field, we can combine pandas str methods with NumPy’s `np.where` function, which is basically a vectorized form of Excel’s IF() macro. It has the following syntax:

`np.where(condition, then, else)`



Here, `condition` is either an array-like object or a Boolean mask. `then` is the value to be used if `condition` evaluates to `True`, and `else` is the value to be used otherwise.

Essentially, `.where()` takes each element in the object used for `condition`, checks whether that particular element evaluates to `True` in the context of the `condition`, and returns an `ndarray` containing `then` or `else`, depending on which applies.

It can be nested into a compound if-then statement, allowing us to compute values based on multiple conditions:

```
np.where(condition1, x1, 
    np.where(condition2, x2, 
        np.where(condition3, x3, ...)))
```

We’ll be making use of these two functions to clean Place of Publication since this column has string objects. Here are the contents of the column:

In [106]:
df['Place of Publication'].head(10)

Identifier
206                                  London
216                London; Virtue & Yorston
218                                  London
472                                  London
480                                  London
481                                  London
519                                  London
667     pp. 40. G. Bryan & Co: Oxford, 1898
874                                 London]
1143                                 London
Name: Place of Publication, dtype: object

We see that for some rows, the place of publication is surrounded by other unnecessary information. If we were to look at more values, we would see that this is the case for only some rows that have their place of publication as ‘London’ or ‘Oxford’.

Let’s take a look at two specific entries:

In [107]:
df.loc[4157862]



Place of Publication                                  Newcastle-upon-Tyne
Date of Publication                                                1867.0
Publisher                                                      T. Fordyce
Title                   Local Records; or, Historical Register of rema...
Author                      FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4157862, dtype: object

In [108]:
df.loc[4159587]

Place of Publication                                  Newcastle upon Tyne
Date of Publication                                                1834.0
Publisher                                                Mackenzie & Dent
Title                   An historical, topographical and descriptive v...
Author                                              Mackenzie, E. (Eneas)
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4159587, dtype: object

These two books were published in the same place, but one has hyphens in the name of the place while the other does not.

To clean this column in one sweep, we can use str.contains() to get a Boolean mask.

We clean the column as follows:



In [112]:
pub = df['Place of Publication']
london = pub.str.contains('London')
london[:5]

Identifier
206    True
216    True
218    True
472    True
480    True
Name: Place of Publication, dtype: bool

In [84]:
oxford = pub.str.contains('Oxford')

We combine them with `np.where`:

In [85]:
df['Place of Publication'] = np.where(london, 'London',
                                      np.where(oxford, 'Oxford',
                                               pub.str.replace('-', ' ')))


Here, the np.where function is called in a nested structure, with condition being a Series of Booleans obtained with str.contains(). The contains() method works similarly to the built-in in keyword used to find the occurrence of an entity in an iterable (or substring in a string).

The replacement to be used is a string representing our desired place of publication. We also replace hyphens with a space with str.replace() and reassign to the column in our DataFrame.

Although there is more dirty data in this dataset, we will discuss only these two columns for now.

Let’s have a look at the first five entries, which look a lot crisper than when we started out:

In [ ]:
df.head()

Things are starting to look cleaner! But let's see if we can improve things furter. 

Do you know what does the `unique` method do? If not Google it! The apply it here to learn about the places of publication in the dataframe.

In [2]:
# print unique values

### ✏️ Now it's your turn. 
> 'Москва, 1860' and 'Moskwa' (i.e. Moscow in English) seem redundant. Apply a str method of your choice among those reviewd above to replace the with the english version 'Moscow'.

# Olympics dataset

### 1.6 Renaming Columns and Skipping Rows
Often, the datasets you’ll work with will have either column names that are not easy to understand, or unimportant information in the first few and/or last rows, such as definitions of the terms in the dataset, or footnotes.

In that case, we’d want to rename columns and skip certain rows so that we can drill down to necessary information with correct and sensible labels.

To demonstrate how we can go about doing this, let’s first take a glance at the initial five rows of the “olympics.csv” dataset:

In [4]:
import pandas as pd

In [ ]:
# load the olympics dataset into a dataframe
df = pd.read_csv('https://raw.githubusercontent.com/realpython/python-data-cleaning/refs/heads/master/Datasets/olympics.csv')
df

This is messy indeed! The columns are the string form of integers indexed at 0. The row which should have been our header (i.e. the one to be used to set the column names) is at `olympics_df.iloc[0]`. This happened because our CSV file starts with 0, 1, 2, …, 15.

Also, if we were to go to the source of this dataset, we’d see that `NaN` above should really be something like “Country”, ? Summer is supposed to represent “Summer Games”, 01 ! should be “Gold”, and so on.

Therefore, we need to do two things:

- Skip one row and set the header as the first (0-indexed) row
- Rename the columns
We can skip rows and set the header while reading the CSV file by passing some parameters to the `read_csv()` function.

This function takes a lot of optional parameters, but in this case we only need one (header) to remove the 0th row:

In [133]:
olympics_df = pd.read_csv('https://raw.githubusercontent.com/realpython/python-data-cleaning/refs/heads/master/Datasets/olympics.csv', header=1)
olympics_df.head()

,Unnamed: 0,? Summer,01 !,02 !,03 !,Total,? Winter,01 !.1,02 !.1,03 !.1,Total.1,? Games,01 !.2,02 !.2,03 !.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


We now have the correct row set as the header and all unnecessary rows removed. Take note of how pandas has changed the name of the column containing the name of the countries from NaN to Unnamed: 0.

To rename the columns, we will make use of a DataFrame’s rename() method, which allows you to relabel an axis based on a mapping (in this case, a dict).

Let’s start by defining a dictionary that maps current column names (as keys) to more usable ones (the dictionary’s values):



In [ ]:
# define a dictionary that maps old column names to new ones


# rename the columns using .rename()

We call the `.rename()` function on our object:

Setting inplace to True specifies that our changes be made directly to the object. Let’s see if this checks out:



### ✏️ Now it's your turn. 
> Read in the olympics dataset and rename the columns as shown above.